In [27]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy.optimize import minimize
from WindPy import w
from sklearn.linear_model import LinearRegression
import datetime
%matplotlib inline

In [28]:
w.start()

.ErrorCode=0
.Data=[Already conntected!]

In [29]:
start_date, end_date = '2018-10-22', datetime.datetime.today()

# 获取股票指数历史收益率

In [30]:
# codes = 'CI005917.WI,CI005919.WI,CI005018.WI,CI005916.WI' # 金融地产、消费、医药、科技
codes = '000934.SH,000931.CSI,000933.SH,000935.SH' # 中证金融地产、可选消费、必选消费、信息技术

In [31]:
data = w.wsd(codes, 'close', start_date, end_date)

In [32]:
df = pd.DataFrame(np.array(data.Data).T, index=data.Times, columns=data.Codes)

In [33]:
rdf = df.pct_change().dropna()

# 获取股票基金历史收益率

In [34]:
fund_df = pd.read_excel(u'../data/股票基金池.xlsx')
# fund_df = pd.read_excel(u'../data/嘉实股票基金池.xlsx')

In [35]:
fund_df

,code,name
0,004450.OF,嘉实前沿科技
1,070021.OF,嘉实主题新动力
2,001637.OF,嘉实腾讯自选股大数据
3,007343.OF,嘉实科技创新
4,001811.OF,中欧明睿新常态A
5,519697.OF,交银优势行业
6,001071.OF,华安媒体互联网
7,519712.OF,交银阿尔法
8,519772.OF,交银新生活力
9,260101.OF,景顺长城优选


In [36]:
codes = fund_df['code'].tolist()

In [37]:
codes

[u'004450.OF',
 u'070021.OF',
 u'001637.OF',
 u'007343.OF',
 u'001811.OF',
 u'519697.OF',
 u'001071.OF',
 u'519712.OF',
 u'519772.OF',
 u'260101.OF',
 u'000978.OF',
 u'519700.OF',
 u'519736.OF',
 u'160918.OF',
 u'001256.OF',
 u'000742.OF',
 u'002229.OF',
 u'001018.OF',
 u'001513.OF']

In [38]:
data = w.wsd(codes, 'nav', start_date, end_date)

In [39]:
fdf = pd.DataFrame(np.array(data.Data).T, index=data.Times, columns=data.Codes)

In [40]:
fdf = fdf.dropna(axis=1)

In [41]:
rfdf = fdf.pct_change().dropna()

In [42]:
codes = rfdf.columns.tolist()

In [43]:
print rfdf.shape
print rdf.shape

(251, 18)
(251, 4)


# 非负线性优化求解

In [44]:
# decomp = pd.DataFrame(index=codes, columns=[u'金融地产', u'消费', u'医药', u'科技'])
decomp = pd.DataFrame(index=codes, columns=[u'金融地产', u'可选消费', u'必选消费', u'科技'])
decomp.index.name = u'基金代码'

In [45]:
def objective(coef, args):
    n = coef.size
    X, y = args[0], args[1]
    target = np.dot(X, coef)
#     error = np.sum(np.square(target - y))
    error = np.sum(np.square(target - y)) + np.linalg.norm(coef, 1) # lasso error
    return error / n

def decomposition(code):
    TOLERANCE = 1e-10
    n = rdf.shape[1]
    init_weights = np.array([1. / n] * n)
    X = rdf.values
    y = rfdf[code].values
    
    constraints = ({'type': 'eq', 'fun': lambda x: np.sum(x) - 1.0},
                   {'type': 'ineq', 'fun': lambda x: x})
    
    opt_res = minimize(fun=objective,
                       x0=init_weights,
                       args=[X, y],
                       constraints=constraints,
                       tol=TOLERANCE,
                       options={'disp': True})
    
    return opt_res.x

In [46]:
decomposition(codes[0])

Optimization terminated successfully.    (Exit mode 0)
            Current function value: 0.252167155961
            Iterations: 14
            Function evaluations: 84
            Gradient evaluations: 14


array([0.09158641, 0.10882472, 0.42542871, 0.37416016])

In [47]:
for c in codes:
    weight = decomposition(c)
    decomp.loc[c] = weight

Optimization terminated successfully.    (Exit mode 0)
            Current function value: 0.252167155961
            Iterations: 14
            Function evaluations: 84
            Gradient evaluations: 14
Optimization terminated successfully.    (Exit mode 0)
            Current function value: 0.25343719591
            Iterations: 15
            Function evaluations: 90
            Gradient evaluations: 15
Optimization terminated successfully.    (Exit mode 0)
            Current function value: 0.251612387583
            Iterations: 9
            Function evaluations: 54
            Gradient evaluations: 9
Optimization terminated successfully.    (Exit mode 0)
            Current function value: 0.252536813469
            Iterations: 15
            Function evaluations: 90
            Gradient evaluations: 15
Optimization terminated successfully.    (Exit mode 0)
            Current function value: 0.252638680904
            Iterations: 14
            Function evaluations: 84
     

In [48]:
decomp

,金融地产,可选消费,必选消费,科技
基金代码,,,,
004450.OF,0.0915864,0.108825,0.425429,0.37416
070021.OF,0.0423589,0.111525,0.165366,0.68075
001637.OF,0.217324,0.229585,0.212082,0.34101
001811.OF,0.170792,0.0754236,0.301407,0.452378
519697.OF,0.151179,0.102674,0.401938,0.344209
001071.OF,0.0632639,-5.20417e-18,0.123021,0.813715
519712.OF,0.133593,0.0533601,0.412149,0.400898
519772.OF,0.146019,0.182761,0.296017,0.375202
260101.OF,0.138488,0.300953,0.308002,0.252557


In [49]:
decomp[u'基金名称'] = fund_df[fund_df['code'].isin(codes)]['name'].tolist()

In [50]:
decomp = decomp[[decomp.columns[-1]] + decomp.columns[:-1].tolist()]

In [51]:
decomp

,基金名称,金融地产,可选消费,必选消费,科技
基金代码,,,,,
004450.OF,嘉实前沿科技,0.0915864,0.108825,0.425429,0.37416
070021.OF,嘉实主题新动力,0.0423589,0.111525,0.165366,0.68075
001637.OF,嘉实腾讯自选股大数据,0.217324,0.229585,0.212082,0.34101
001811.OF,中欧明睿新常态A,0.170792,0.0754236,0.301407,0.452378
519697.OF,交银优势行业,0.151179,0.102674,0.401938,0.344209
001071.OF,华安媒体互联网,0.0632639,-5.20417e-18,0.123021,0.813715
519712.OF,交银阿尔法,0.133593,0.0533601,0.412149,0.400898
519772.OF,交银新生活力,0.146019,0.182761,0.296017,0.375202
260101.OF,景顺长城优选,0.138488,0.300953,0.308002,0.252557


In [52]:
decomp.to_excel(u'../data/股票基金收益率分解.xlsx')
# decomp.to_excel(u'../data/嘉实股票基金收益率分解.xlsx')